# Import Useful Libraries

In [1]:
import pandas as pd
from stringmatch.model import *

ModuleNotFoundError: No module named 'stringmatch'

# Load Data

In [2]:
df_all_versions_cols = pd.read_excel('all_versions_of_columns.xlsx')

In [3]:
df_all_versions_cols

,Athena Non-GPO,Calculated Non-GPO,Calculated GPO,Athena GPO
0,dk01,dl12.reg_number,reg_number,reg_number
1,dk02,dl12.commercial_name,commercial_name,commercial_name
2,dk03,dl12.identification.legal_status,identification.legal_status,identification.legal_status
3,dk04,dl12.identification.incorporation_date,identification.incorporation_date,identification.incorporation_date
4,dk10.cicsregion,dl12.identification.dissolution_date,identification.dissolution_date,identification.registered_office.location_line1
...,...,...,...,...
1640,NaN,dld6.ltd_company_cais.ltd_company_cais_details...,ltd_company_cais.ltd_company_cais_details.comm...,NaN
1641,NaN,dld6.ltd_company_cais.ltd_company_cais_details...,ltd_company_cais.ltd_company_cais_details.comm...,NaN
1642,NaN,err1.error.error_code,error.error_code,NaN
1643,NaN,err1.error.severity,error.severity,NaN


In [4]:
df_athena_gpo = pd.DataFrame(df_all_versions_cols['Athena GPO'].dropna())
df_calculated_cols = pd.DataFrame(df_all_versions_cols[['Calculated Non-GPO', 'Calculated GPO']])
df_athena_gpo = pd.DataFrame(df_all_versions_cols[['Athena GPO']])

In [5]:
df_athena_gpo

,Athena GPO
0,reg_number
1,commercial_name
2,identification.legal_status
3,identification.incorporation_date
4,identification.registered_office.location_line1
...,...
1640,NaN
1641,NaN
1642,NaN
1643,NaN


# Make Predictions

In [6]:
matches = find_match_for_list(df_all_versions_cols['Athena Non-GPO'].dropna(),
                              df_all_versions_cols['Calculated Non-GPO'],
                              function='damerau-levenshtein',
                              weights=(8, 1, 10, 9))

In [7]:
df_matches = pd.DataFrame(matches, columns=['Athena Non-GPO', 'Calculated Non-GPO', 'Score'])

In [8]:
df_matches

,Athena Non-GPO,Calculated Non-GPO,Score
0,dk01,dl12.reg_number,157
1,dk02,dl12.reg_number,157
2,dk03,dl12.reg_number,166
3,dk04,dl12.reg_number,166
4,dk10.cicsregion,dl80.scoring_data.region,85
...,...,...,...
885,dl60.adminiraddr4,dl60.legal_notices.administrators.discharge.ad...,73
886,dl60.adminiraddr5,dl60.legal_notices.administrators.discharge.ad...,73
887,processed_year,dl41.payment_performance.payment_full.ind_paym...,109
888,processed_month,dl41.payment_performance.payment_full.fastest_...,94


In [12]:
df_merged = df_matches.merge(df_calculated_cols, on='Calculated Non-GPO', how='left')\
                      .merge(df_athena_gpo, left_on='Calculated GPO', right_on='Athena GPO', how='left')

In [13]:
df_merged

,Athena Non-GPO,Calculated Non-GPO,Score,Calculated GPO,Athena GPO
0,dk01,dl12.reg_number,157,reg_number,NaN
1,dk02,dl12.reg_number,157,reg_number,NaN
2,dk03,dl12.reg_number,166,reg_number,NaN
3,dk04,dl12.reg_number,166,reg_number,NaN
4,dk10.cicsregion,dl80.scoring_data.region,85,scoring_data.region,NaN
...,...,...,...,...,...
885,dl60.adminiraddr4,dl60.legal_notices.administrators.discharge.ad...,73,legal_notices.administrators.discharge.adminis...,NaN
886,dl60.adminiraddr5,dl60.legal_notices.administrators.discharge.ad...,73,legal_notices.administrators.discharge.adminis...,NaN
887,processed_year,dl41.payment_performance.payment_full.ind_paym...,109,payment_performance.payment_full.ind_payment_p...,payment_performance.payment_full.ind_payment_p...
888,processed_month,dl41.payment_performance.payment_full.fastest_...,94,payment_performance.payment_full.fastest_month,payment_performance.payment_full.fastest_month


In [20]:
print('Number of estimate (true and false pos): ', df_merged['Athena GPO'].isnull().sum())
print('Percentage of non-null: {:.1%}'.format(1 - (df_merged['Athena GPO'].isnull().sum()/df_matches.shape[0])))

Number of estimate (true and false pos):  290
Percentage of non-null: 67.4%


In [16]:
# df_merged.to_csv('column_match_predictions.csv', index=False)

# Calculate accuracy

In [66]:
df_col_match_pred = pd.read_csv('column_match_predictions.csv')
new = df_col_match_pred['Athena Non-GPO'].str.split('.', n = 1, expand = True)
new[0] = new[0].str.strip()
df_col_match_pred['Table Code'] = new[0]
df_col_match_pred['Table Name'] = new[1]

codes_to_lose = ['dk01', 'dk02', 'dk03', 'dk04', 'dk10', 'dk12', 'dl01', 'dl10', 'dl19', 'dl38', 'dl87',\
                 'dk11', 'dk13', 'dl25', 'dl39', 'dr50', 'kptd']
# df[~df.country.isin(countries_to_keep)]
df_col_match_pred_cleaned = df_col_match_pred[~df_col_match_pred['Table Code'].isin(codes_to_lose)]

In [68]:
df_col_match_pred_cleaned

,Athena Non-GPO,Calculated Non-GPO,Calculated GPO,Athena GPO,Status,Table Code,Table Name
15,dl12.companyname,dl12.commercial_name,commercial_name,commercial_name,d,dl12,companyname
16,dl12.datedissvd_dd,dl12.identification.dissolution_date,identification.dissolution_date,identification.dissolution_date,d,dl12,datedissvd_dd
17,dl12.datedissvd_mm,dl14.identification.previous_names.previous_na...,identification.previous_names.previous_name.name,identification.dissolution_date,d,dl12,datedissvd_mm
18,dl12.datedissvd_yyyy,dl12.identification.dissolution_date,identification.dissolution_date,identification.dissolution_date,d,dl12,datedissvd_yyyy
19,dl12.dateincorp_dd,dl12.identification.incorporation_date,identification.incorporation_date,identification.incorporation_date,d,dl12,dateincorp_dd
...,...,...,...,...,...,...,...
885,dl60.adminiraddr4,dl60.legal_notices.administrators.discharge.ad...,legal_notices.administrators.discharge.adminis...,?,d,dl60,adminiraddr4
886,dl60.adminiraddr5,dl60.legal_notices.administrators.discharge.ad...,legal_notices.administrators.discharge.adminis...,?,d,dl60,adminiraddr5
887,processed_year,dl41.payment_performance.payment_full.ind_paym...,payment_performance.payment_full.ind_payment_p...,?,d,processed_year,None
888,processed_month,dl41.payment_performance.payment_full.fastest_...,payment_performance.payment_full.fastest_month,?,d,processed_month,None


In [69]:
y_pred = df_col_match_pred['Calculated GPO']
y_true = df_col_match_pred['Athena GPO']
y_pred_cleaned = df_col_match_pred_cleaned['Calculated GPO']
y_true_cleaned = df_col_match_pred_cleaned['Athena GPO']

In [71]:
print('Accuracy All: {:.1%}'.format((y_pred == y_true).sum() / y_true.shape[0]))
print('Accuracy Cleaned: {:.1%}'.format((y_pred_cleaned == y_true_cleaned).sum() / y_true_cleaned.shape[0]))

Accuracy All: 43.9%
Accuracy Cleaned: 49.9%


In [72]:
y_true.shape[0]

890

In [73]:
y_true_cleaned.shape[0]

782